<a href="https://colab.research.google.com/github/StetskoSergey/pida5_stetsko/blob/master/aml6%20dz%201.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Строим логистическую регрессию - угадываем пол спортсмена по признакам

https://www.kaggle.com/rio2016/olympic-games

In [0]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv('adult.csv')
data.head()

,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0.0,0.0,40.0,United-States,<=50K
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0.0,0.0,50.0,United-States,<=50K
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0.0,0.0,40.0,United-States,>50K
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688.0,0.0,40.0,United-States,>50K
4,18,?,103497,Some-college,10,Never-married,?,Own-child,White,Female,0.0,0.0,30.0,United-States,<=50K


In [3]:
data.shape

(38464, 15)

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38464 entries, 0 to 38463
Data columns (total 15 columns):
age                38464 non-null int64
workclass          38464 non-null object
fnlwgt             38464 non-null int64
education          38464 non-null object
educational-num    38464 non-null int64
marital-status     38464 non-null object
occupation         38464 non-null object
relationship       38463 non-null object
race               38463 non-null object
gender             38463 non-null object
capital-gain       38463 non-null float64
capital-loss       38463 non-null float64
hours-per-week     38463 non-null float64
native-country     38463 non-null object
income             38463 non-null object
dtypes: float64(3), int64(3), object(9)
memory usage: 4.4+ MB


#### Список всех категориальных признаков

In [0]:
list_col = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'gender', 'native-country']

Посмотрим много ли в наших признаках пустых значений

In [16]:
for col in list_col:
  print ('Для {} пустых строк {}'.format(col, len(data[pd.isnull(data[col])])))
  print ('Для {} уникальных значений {}'.format(col, len(data[col].unique())))

print ('Всего строк в наборе {}'.format(len(data)))

Для workclass пустых строк 0
Для workclass уникальных значений 9
Для education пустых строк 0
Для education уникальных значений 16
Для marital-status пустых строк 0
Для marital-status уникальных значений 7
Для occupation пустых строк 0
Для occupation уникальных значений 16
Для relationship пустых строк 1
Для relationship уникальных значений 7
Для race пустых строк 1
Для race уникальных значений 6
Для gender пустых строк 1
Для gender уникальных значений 3
Для native-country пустых строк 1
Для native-country уникальных значений 43
Для income пустых строк 1
Для income уникальных значений 3
Всего строк в наборе 38464


In [0]:
# удаляем ячейки с отсутствием данных
data_n = data.dropna()
Y = data_n['income']
data_n = data_n.drop( columns = 'income')

In [0]:
from sklearn.linear_model import LogisticRegression

In [40]:
# попробуем выбрать какие-нибудь признаки

# переведем категориальные переменные в 0,1 
X = pd.get_dummies(data_n, columns = list_col)

X.head()

,age,fnlwgt,educational-num,capital-gain,capital-loss,hours-per-week,workclass_?,workclass_Federal-gov,workclass_Local-gov,workclass_Never-worked,workclass_Private,workclass_Self-emp-inc,workclass_Self-emp-not-inc,workclass_State-gov,workclass_Without-pay,education_10th,education_11th,education_12th,education_1st-4th,education_5th-6th,education_7th-8th,education_9th,education_Assoc-acdm,education_Assoc-voc,education_Bachelors,education_Doctorate,education_HS-grad,education_Masters,education_Preschool,education_Prof-school,education_Some-college,marital-status_Divorced,marital-status_Married-AF-spouse,marital-status_Married-civ-spouse,marital-status_Married-spouse-absent,marital-status_Never-married,marital-status_Separated,marital-status_Widowed,occupation_?,occupation_Adm-clerical,...,native-country_Canada,native-country_China,native-country_Columbia,native-country_Cuba,native-country_Dominican-Republic,native-country_Ecuador,native-country_El-Salvador,native-country_England,native-country_France,native-country_Germany,native-country_Greece,native-country_Guatemala,native-country_Haiti,native-country_Holand-Netherlands,native-country_Honduras,native-country_Hong,native-country_Hungary,native-country_India,native-country_Iran,native-country_Ireland,native-country_Italy,native-country_Jamaica,native-country_Japan,native-country_Laos,native-country_Mexico,native-country_Nicaragua,native-country_Outlying-US(Guam-USVI-etc),native-country_Peru,native-country_Philippines,native-country_Poland,native-country_Portugal,native-country_Puerto-Rico,native-country_Scotland,native-country_South,native-country_Taiwan,native-country_Thailand,native-country_Trinadad&Tobago,native-country_United-States,native-country_Vietnam,native-country_Yugoslavia
0,25,226802,7,0.0,0.0,40.0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
1,38,89814,9,0.0,0.0,50.0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
2,28,336951,12,0.0,0.0,40.0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,44,160323,10,7688.0,0.0,40.0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
4,18,103497,10,0.0,0.0,30.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0


In [0]:
# целевая переменная (столбец sex) снова является категориальной
# переведем значения столбца в числа, оставив один столбец

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

In [42]:
le.fit(Y)

LabelEncoder()

In [35]:
le.classes_

array(['<=50K', '>50K'], dtype=object)

In [46]:
# записываем в переменную y преобразованный столбец
y = pd.Series(data = le.transform(Y))
y.head()

0    0
1    0
2    1
3    1
4    0
dtype: int64

In [0]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#### Сделаем нормализацию данных, это позволит в 2 раза увеличить точность модели

In [0]:
from sklearn.preprocessing import StandardScaler 
scale_features_std = StandardScaler() 
features_train = scale_features_std.fit_transform(X_train) 
features_test = scale_features_std.transform(X_test) 

In [0]:
model = LogisticRegression(C = 1.0, solver='lbfgs') #тут надо играть с C

In [0]:
# обучаем модель

model.fit(features_train, y_train)
y_pred = model.predict(features_test) ## но есть еще и predict_proba, уже знакомая вам по деревьям

In [89]:
from sklearn.metrics import classification_report
print (classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.88      0.94      0.91      5874
           1       0.75      0.60      0.67      1819

    accuracy                           0.86      7693
   macro avg       0.82      0.77      0.79      7693
weighted avg       0.85      0.86      0.85      7693



баланс между полнотой и точностью решается за счет predict_proba (решайте сами, какие ошибки вам дороже...)

In [0]:
y_pred_proba = model.predict_proba(features_test).tolist()

In [91]:
y_pred_proba[0]

[0.7746826011362539, 0.22531739886374613]

In [92]:
threshold = 0.41 ## играйте с threshold
y_pred_2 = [1 if y[1] > threshold else 0 for y in y_pred_proba]
print (classification_report(y_test, y_pred_2))

              precision    recall  f1-score   support

           0       0.90      0.91      0.91      5874
           1       0.70      0.68      0.69      1819

    accuracy                           0.86      7693
   macro avg       0.80      0.79      0.80      7693
weighted avg       0.85      0.86      0.85      7693



#### C 90% точностью можно предсказать "бедность" и 70% точностью "богатство"